In [4]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)

In [5]:
def format_value(value):
  if value == '.':
    return '..'
  else:
    s = str(value)
    if len(s) == 1:
      s = ' ' + s
    return s

def format_h(const, max):
  s = ''
  for i in range(max - len(const)):
    s = s + '   '
  s = s + ' '.join(map(format_value, const))
  return s

def pretty_print(board, h_const, v_const):
  max_h = 1
  for c in h_const:
    max_h = max(max_h, len(c))
  max_v = 1
  for c in v_const:
    max_v = max(max_v, len(c))

  h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
  for i in range(max_v):
    print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), v_const)))
  for r in range(len(board)):
    row = board[r]
    print((h_offset if r ==0 else format_h(h_const[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

In [6]:
problem = get_problem('kakuro')

pprint.pprint(problem)

board = problem['board']
h_const = problem['h_constraints']
v_const = problem['v_constraints']

print('Input puzzle')
pretty_print(board, h_const, v_const)

# This does not solve the problem, just makes it the same as the input
solution = board

# We print the solution in a way to make it easy to debug and check
print('Solution')
pretty_print(solution, h_const, v_const)

answer = {
    'board': board,
    'h_const': h_const,
    'v_const': v_const,
    'id': problem['id']
}

# The answer is submitted to the server, this is just to check the format (it does not check the solution itself)
# Note: We are doing this just to get used to it in following exercises mostly
submit_answer('kakuro', answer)

{'board': [['X', 'X', 'X', 'X', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', '.', '.', '.', 'X'],
           ['X', 'X', '.', '.', '.'],
           ['X', 'X', '.', '.', '.']],
 'h_constraints': [[7], [19], [8], [15]],
 'id': 0,
 'v_constraints': [[5], [12], [22], [10]]}
Input puzzle
       5 12 22 10
    X  X  X  X  X
 7  X .. .. ..  X
19  X .. .. ..  X
 8  X  X .. .. ..
15  X  X .. .. ..
Solution
       5 12 22 10
    X  X  X  X  X
 7  X .. .. ..  X
19  X .. .. ..  X
 8  X  X .. .. ..
15  X  X .. .. ..
Ignore: not checking values yet


In [10]:
# representation
class Kakuro:
    def __init__(self, puzzle):
        self.board = puzzle['board']
        self.h_constraints = puzzle['h_constraints']
        self.v_constraints = puzzle['v_constraints']
        self.id = puzzle.get('id', None)

    def pretty_print(self):
        max_h = 1
        for c in self.h_constraints:
            max_h = max(max_h, len(c))
        max_v = 1
        for c in self.v_constraints:
            max_v = max(max_v, len(c))

        h_offset = ' '.join(map(lambda x: '  ', range(max_h)))
        for i in range(max_v):
            print('    ' + h_offset + ' '.join(map(lambda x: '  ' if i >= len(x) else format_value(x[i]), self.v_constraints)))
        for r in range(len(self.board)):
            row = self.board[r]
            print((h_offset if r ==0 else format_h(self.h_constraints[r-1], max_h)) + ' ' +  ' '.join(map(format_value, row)))

    def is_valid(self):
        # Check horizontal constraints
        for r, row in enumerate(self.board):
            for c, value in enumerate(row):
                if value != 'X' and value != '.':
                    # Check the sum of horizontal "word"
                    if not self.check_horizontal(r, c):
                        return False
                    # Check the sum of vertical "word"
                    if not self.check_vertical(r, c):
                        return False
        return True
    
    def check_horizontal(self, row, col):
        target_sum = self.h_constraints[row-1][0]
        total, count = 0, 0
        for c in range(col, len(self.board[row])):
            cell = self.board[row][c]
            if cell == 'X':  # End of the horizontal "word"
                break
            if cell == '.':  # Empty cell, not yet assigned
                continue
            total += int(cell)
            count += 1
        if count == 0:  # No cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded
        if 'X' not in self.board[row][col:] and total != target_sum:
            return False  # Reached end of "word" and sum does not match
        return True

    def check_vertical(self, row, col):
        target_sum = self.v_constraints[col-1][0]
        total, count = 0, 0
        for r in range(row, len(self.board)):
            cell = self.board[r][col]
            if cell == 'X':  # End of the vertical "word"
                break
            if cell == '.':  # Empty cell, not yet assigned
                continue
            total += int(cell)
            count += 1
        if count == 0:  # No cells filled in yet
            return True
        if total > target_sum:
            return False  # Sum exceeded
        if 'X' not in [self.board[r][col] for r in range(row, len(self.board))] and total != target_sum:
            return False  # Reached end of "word" and sum does not match
        return True




In [15]:
# To solve the given problem, we'll complete the 'get_possible_values' function.
# This function will check the constraints and return possible values for the cell at (row, col).

def get_possible_values(board, row, col, h_constraints, v_constraints):
    # Possible values are 1 through 9
    possible_values = set(range(1, 10))

    # Find the start of the horizontal "word"
    h_start = col
    while h_start > 0 and board[row][h_start - 1] != 'X':
        h_start -= 1
    # Find the end of the horizontal "word"
    h_end = col
    while h_end < len(board[row]) and board[row][h_end] != 'X':
        h_end += 1
    h_word = board[row][h_start:h_end]
    h_target_sum = h_constraints[row - 1][0] if row > 0 else None

    # Find the start of the vertical "word"
    v_start = row
    while v_start > 0 and board[v_start - 1][col] != 'X':
        v_start -= 1
    # Find the end of the vertical "word"
    v_end = row
    while v_end < len(board) and board[v_end][col] != 'X':
        v_end += 1
    v_word = [board[i][col] for i in range(v_start, v_end)]
    v_target_sum = v_constraints[col - 1][0] if col > 0 else None

    # Remove values from possible_values that are already used in the horizontal and vertical "words"
    for value in h_word + v_word:
        if value != '.' and value in possible_values:
            possible_values.remove(value)

    # Further filter the possible_values based on the sum constraints
    if h_target_sum is not None:
        h_sum = sum(int(v) for v in h_word if v != '.')
        h_possible_values = set(val for val in possible_values if h_sum + val <= h_target_sum)
        possible_values.intersection_update(h_possible_values)
    
    if v_target_sum is not None:
        v_sum = sum(int(v) for v in v_word if v != '.')
        v_possible_values = set(val for val in possible_values if v_sum + val <= v_target_sum)
        possible_values.intersection_update(v_possible_values)

    return possible_values

# Correcting the logic for traversing the board to find the next empty cell.

def solve_kakuro(board, h_constraints, v_constraints):
    # Initialize the stack with a tuple containing the board and the starting cell
    stack = [(board, 0, 0)]
    backtracks = 0  # To count the number of backtracks

    while stack:
        board, row, col = stack.pop()  # Get the current state and cell to fill

        # Find the next empty cell or backtrack if needed
        found_empty = False
        while not found_empty and row < len(board):
            if board[row][col] == '.':
                found_empty = True
            else:
                col += 1
                if col == len(board[row]):  # Move to the next row
                    col = 0
                    row += 1

        if not found_empty:  # All cells are filled or need to backtrack
            if row == len(board):
                return board, backtracks  # All cells are filled
            backtracks += 1  # Increment backtracks if we need to backtrack
            continue

        # Get possible values for the current cell
        possible_values = get_possible_values(board, row, col, h_constraints, v_constraints)

        # If there are no possible values, we need to backtrack
        if not possible_values:
            backtracks += 1
            continue  # Backtrack to the previous state

        # Try each possible value and push the new state onto the stack
        for value in possible_values:
            new_board = [list(row) for row in board]  # Create a deep copy of the board
            new_board[row][col] = value
            stack.append((new_board, row, col))  # Push the new state

    # If the stack is empty, no solution was found
    return None, backtracks





In [18]:
problem = {
    'board': [['X', 'X', 'X'],
              ['X', '.', '.'],
              ['X', '.', '.']],
    'h_constraints': [[12], [6]],
    'id': 1,
    'v_constraints': [[10], [8]]
}

# print the initial board
pretty_print(problem['board'], problem['h_constraints'], problem['v_constraints'])

# Now, let's solve the puzzle again with the refined 'get_possible_values' function
solution, backtracks = solve_kakuro(problem['board'], problem['h_constraints'], problem['v_constraints'])
solution, backtracks

# print the solution
pretty_print(solution, problem['h_constraints'], problem['v_constraints'])

      10  8
    X  X  X
12  X .. ..
 6  X .. ..
      10  8
    X  X  X
12  X  9  3
 6  X  1  5
